In [1]:
%matplotlib qt

In [2]:
import hyperspy.api as hs
from glob import glob 
from copy import copy 
import gc
import os
from tqdm import tqdm_notebook
import png

In [3]:
def collective_max_pixel_spectrum(bcf_file_list, **kwargs):
    """calculate and return the max_pixel_spectrum for whole list of bcf files"""
    eds_list = []
    for i in tqdm_notebook(bcf_file_list):
        #print(' '.join([str(bcfs.index(i)),'from',str(len(bcf_file_list))]))
        EDS = hs.load(i, select_type='spectrum', **kwargs)
        eds_list.append(copy(EDS.max()))
        del EDS
        gc.collect()
    stacky = hs.stack(eds_list)
    max_pixel_spectra = stacky.max(axis=0)
    return max_pixel_spectra
    

In [4]:
def png_exporter(bcf_file_list, line_list, dtype='uint8', save_electron_images=False,
                integration_windows=1.5):
    """extract and save the images of electron images and selected x-ray lines to png files
    arguments:
    bcf_file_list : the python list with relative or absolute paths to bcf files
    line_list - python list with x-ray element lines for extracting (n.p. 'Si_Ka')
    dtype - by default uint8, use uint32 if it is intense
    """
    for i in tqdm_notebook(bcf_file_list, desc='total bcf'):
        # all signals
        all_signals = hs.load(i)
        basename = i.rsplit('.',1)[0]
        if type(all_signals) == list:
            if save_electron_images:
                for k in all_signals[:-1]:
                    signal_name = k.metadata.Signal.signal_type 
                    img = png.from_array(k.data, 'L')
                    img.save('_'.join([basename, signal_name + '.png']))
            el_maps = all_signals[-1].get_lines_intensity(xray_lines=line_list,
                                                          integration_windows=integration_windows)
        else:
            el_maps = all_signals.get_lines_intensity(xray_lines=line_list,
                                                   integration_windows=integration_windows)
        for j in tqdm_notebook(range(len(line_list)), desc='elements_at_bcf', leave=False):
            el_maps[j].change_dtype(dtype)
            img = png.from_array(el_maps[j].data, 'L')  # 'L' bo szaro-bialy
            img.save('_'.join([basename, line_list[j].replace('_','-') + '.png']))
        del all_signals, img, el_maps
        gc.collect()

#### This is where customisation starts:

In [5]:
bcfs = glob('*.bcf')  # add path to your bcf files

In [6]:
bcfs[:5]  # check the list is not empty

[]

#### If we don't know what lines interest us or what to except in the sample, then we should do 'max pixel spectra' for whole set of data:

In [9]:
max_pixel_spectrum = collective_max_pixel_spectrum(bcfs, downsample=4,
                                                   cutoff_at_kV=13)

In [36]:
max_pixel_spectrum

<EDSSEMSpectrum, title: Stack of EDX, dimensions: (|1347)>

In [7]:
max_pixel_spectrum.plot()

NameError: name 'max_pixel_spectrum' is not defined

In [33]:
#select or refine the lines until all major peaks in the max_pixel spectra are covered
lines = ['C_Ka', 'O_Ka', 'F_Ka', 'Na_Ka', 'Mg_Ka',
         'Al_Ka', 'Si_Ka', 'P_Ka', 'Zr_La', 'Nb_La',
         'K_Ka', 'Ca_Ka', 'Ti_Ka', 'Cr_Ka','Fe_Ka',
         'Ni_Ka', 'S_Ka', 'Cl_Ka', 'La_La', 'Ce_La',
         'Nd_La','Cu_Ka', 'Zn_Ka', 'Au_La', 'Th_Ma']
len(lines)

25

In [37]:
max_pixel_spectrum.plot(xray_lines=lines)

CPU times: user 2min 5s, sys: 2.54 s, total: 2min 7s
Wall time: 2min 5s


In [25]:
#lets save the pictures as png
png_exporter(bcfs, lines, save_electron_images=True)